# `imbrium` use case
## Hyper parameter optimization - Bidirectional Long Short-term Neural Network

### Multivariate pure forecasting

##### Example Steps:

- basic data preparation
- scale target and feature numpy arrays
- create imbrium bidirectional long short-term neural network
- use optuna to perform hyper parameter optimzation on shifting window variables

In [1]:
import imbrium
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import optuna

print(f"imbrium version: {imbrium.__version__} loaded")

imbrium version: 3.0.0 loaded


In [2]:
example_data = pd.read_csv('example_dataset/AirQualityUCI.csv', delimiter=';')
example_data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [3]:
example_data = example_data[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [4]:
example_data.isna().sum()

CO(GT)           114
PT08.S1(CO)      114
NMHC(GT)         114
C6H6(GT)         114
PT08.S2(NMHC)    114
dtype: int64

In [5]:
example_data = example_data.dropna()

In [6]:
example_data.notna().sum()

CO(GT)           9357
PT08.S1(CO)      9357
NMHC(GT)         9357
C6H6(GT)         9357
PT08.S2(NMHC)    9357
dtype: int64

In [7]:
example_data = example_data.replace(",", ".", regex = True).astype("float")

In [8]:
target = np.array(example_data['PT08.S1(CO)']).reshape(-1, 1)

In [9]:
features = example_data[['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [10]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

scaled_features = feature_scaler.fit_transform(features)
scaled_target = target_scaler.fit_transform(target)

In [11]:
custom_layer_config = {
            "layer0": {
                "config": {
                    "neurons": 200,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer1": {
                "config": {
                    "neurons": 100,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer2": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.002
                }
            },
            "layer4": {
                "config": {
                    "neurons": 25,
                    "activation": "relu",
                    "regularization": 0.002,
                }
            },
        }

In [12]:
from imbrium import PureMulti

In [13]:
predictor_instance = PureMulti(target = scaled_target, features = scaled_features)


def objective(trial):

    steps_past = trial.suggest_int('steps_past', 5, 30)
    steps_future = trial.suggest_int('steps_future', 5, 10)
    
    
    predictor_instance.create_fit_bilstm(
        steps_past =  steps_past,
        steps_future = steps_future,
        loss='mean_squared_error',
        metrics='mean_squared_error',
        bilstm_block_one = 3,
        lstm_block_one = 2,
        layer_config = custom_layer_config, 
        epochs=10,
        show_progress=1,
        board=False,
        monitor='val_loss',
        patience=3,
        min_delta=0,
        verbose=1
    )

    predictor_instance.evaluate_model()
    metric_value = predictor_instance.show_evaluation()[1]

    return metric_value

study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=5) 

[I 2024-05-14 01:31:03,528] A new study created in memory with name: no-name-c25ee50c-e45c-46fd-9292-85e57731d31d



Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - loss: 1.3796 - mean_squared_error: 0.1406 - val_loss: 0.1532 - val_mean_squared_error: 0.0295
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - loss: 0.1195 - mean_squared_error: 0.0200 - val_loss: 0.0878 - val_mean_squared_error: 0.0329
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - loss: 27.3251 - mean_squared_error: 27.2698 - val_loss: 0.2643 - val_mean_squared_error: 0.0404
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 16s 85ms/step - loss: 0.2448 - mean_squared_error: 0.0235 - val_loss: 0.2481 - val_mean_squared_error: 0.0334
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - loss: 0.2331 - mean_squared_error: 0.0197 - val_loss: 0.2422 - val_mean_squared_error: 0.0321
Epoch 5: early stopping
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.2330 - mean_squared_error: 0.0228


[I 2024-05-14 01:32:29,789] Trial 0 finished with value: 0.02011769823729992 and parameters: {'steps_past': 20, 'steps_future': 8}. Best is trial 0 with value: 0.02011769823729992.


Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 42s 145ms/step - loss: 1.5477 - mean_squared_error: 0.1867 - val_loss: 0.3769 - val_mean_squared_error: 0.0336
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 24s 128ms/step - loss: 0.3270 - mean_squared_error: 0.0204 - val_loss: 0.2610 - val_mean_squared_error: 0.0337
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 23s 126ms/step - loss: 0.2308 - mean_squared_error: 0.0203 - val_loss: 0.2001 - val_mean_squared_error: 0.0317
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 20s 108ms/step - loss: 0.1775 - mean_squared_error: 0.0194 - val_loss: 0.1603 - val_mean_squared_error: 0.0294
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 17s 93ms/step - loss: 0.1425 - mean_squared_error: 0.0187 - val_loss: 0.1369 - val_mean_squared_error: 0.0316
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 17s 89ms/step - loss: 0.1191 - mean_squared_error: 0.0193 - val_loss: 0.1138 - val_mean_squared_error: 0.0288
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 18s 94ms/step - loss: 0.1258 - mean_squared_error: 0.044

[I 2024-05-14 01:36:05,087] Trial 1 finished with value: 0.020822398364543915 and parameters: {'steps_past': 22, 'steps_future': 8}. Best is trial 0 with value: 0.02011769823729992.


...

[I 2024-05-14 01:39:22,905] Trial 2 finished with value: 0.01466854102909565 and parameters: {'steps_past': 22, 'steps_future': 8}. Best is trial 2 with value: 0.01466854102909565.


Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 27s 90ms/step - loss: 1.3462 - mean_squared_error: 0.1336 - val_loss: 0.1227 - val_mean_squared_error: 0.0291
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 16s 85ms/step - loss: 0.0943 - mean_squared_error: 0.0202 - val_loss: 0.0722 - val_mean_squared_error: 0.0326
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 15s 80ms/step - loss: 683.5151 - mean_squared_error: 683.4765 - val_loss: 9594.3340 - val_mean_squared_error: 9594.0264
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 3366.2937 - mean_squared_error: 3365.9736 - val_loss: 0.6143 - val_mean_squared_error: 0.2981
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 15s 80ms/step - loss: 54.7801 - mean_squared_error: 54.4653 - val_loss: 0.5971 - val_mean_squared_error: 0.2857
Epoch 5: early stopping
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.5731 - mean_squared_error: 0.2617


[I 2024-05-14 01:40:53,150] Trial 3 finished with value: 0.26853612065315247 and parameters: {'steps_past': 17, 'steps_future': 8}. Best is trial 2 with value: 0.01466854102909565.


Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 36s 142ms/step - loss: 1.6537 - mean_squared_error: 0.1986 - val_loss: 0.5017 - val_mean_squared_error: 0.0325
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 24s 128ms/step - loss: 0.4454 - mean_squared_error: 0.0207 - val_loss: 0.3576 - val_mean_squared_error: 0.0322
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 26s 138ms/step - loss: 0.3234 - mean_squared_error: 0.0197 - val_loss: 0.2804 - val_mean_squared_error: 0.0326
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 25s 134ms/step - loss: 0.2537 - mean_squared_error: 0.0198 - val_loss: 0.2301 - val_mean_squared_error: 0.0326
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step - loss: 0.2069 - mean_squared_error: 0.0199 - val_loss: 0.1912 - val_mean_squared_error: 0.0325
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 24s 128ms/step - loss: 0.1709 - mean_squared_error: 0.0198 - val_loss: 0.1609 - val_mean_squared_error: 0.0311
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 24s 128ms/step - loss: 0.1433 - mean_squared_error: 0.

[I 2024-05-14 01:45:12,294] Trial 4 finished with value: 0.019185613840818405 and parameters: {'steps_past': 28, 'steps_future': 10}. Best is trial 2 with value: 0.01466854102909565.
